In [1]:
import os
import os.path as op

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder


In [ ]:
WORKING_PATH = "..."  # Set your working path here.

beh = pd.read_csv(
    op.join(WORKING_PATH, "data/beh/beh_residualized_extended_050225.csv")
)
aparc = pd.read_csv(
    op.join(WORKING_PATH, "data/fs-measures/aparcstats2table_combined.csv")
)
aseg = pd.read_csv(
    op.join(WORKING_PATH, "data/fs-measures/aseg_stats_combined.csv")
).rename(columns={"Measure:volume": "id"})

beh = beh[beh["site"] == "regensburg"]
db = pd.merge(beh, aparc, on="id", how="inner").merge(aseg, on="id", how="inner")
db

In [5]:
def plot_brain_correlations(data, x, y, y_label, out_path=None):
    # Set style and color palette
    sns.set_style(
        "whitegrid",
        {"axes.grid": False, "xtick.bottom": False, "ytick.left": False},
    )
    palette = {"male": "#2E86AB", "female": "#F24236"}

    # Create figure
    plt.figure(figsize=(12, 11), dpi=300)

    # Create jointplot with enhanced styling
    g = sns.jointplot(
        data=data,
        x=x,
        y=y,
        hue="sex",
        kind="scatter",
        height=10,
        ratio=8,
        palette=palette,
        marginal_kws=dict(common_norm=False, fill=True),
        joint_kws=dict(alpha=0.7, s=100),
    )

    # Add regression lines with confidence intervals
    for sex_group in data["sex"].unique():
        subset = data[data["sex"] == sex_group]
        sns.regplot(
            data=subset,
            x=x,
            y=y,
            scatter=False,
            line_kws={"linestyle": "--", "linewidth": 2},
            ci=95,
            ax=g.ax_joint,
            color=palette[sex_group],
        )

    # Add overall regression line
    sns.regplot(
        data=data,
        x=x,
        y=y,
        scatter=False,
        color="black",
        line_kws={"linewidth": 2},
        ci=95,
        ax=g.ax_joint,
    )

    # Customize labels and appearance
    g.ax_joint.set_xlabel(
        "Cortisol increase (nmol/l)".title(), fontsize=20, fontweight="bold"
    ).set_visible(False)  ## To set the visibility off.
    g.ax_joint.set_ylabel(y_label, fontsize=20, fontweight="bold").set_visible(
        False
    )  ## To set the visibility off.
    g.ax_joint.tick_params(labelsize=18)
    # Remove axis values
    # g.ax_joint.set_xticklabels([])
    # g.ax_joint.set_yticklabels([])

    # Customize legend
    g.ax_joint.legend(
        title="Sex",
        title_fontsize=12,
        fontsize=11,
        bbox_to_anchor=(0.95, 0.15),
        frameon=True,
        edgecolor="black",
    ).set_visible(False)

    # Make axes thicker
    g.ax_joint.spines["bottom"].set_linewidth(2)
    g.ax_joint.spines["left"].set_linewidth(2)
    g.ax_joint.tick_params(width=2)

    # Adjust layout and save
    plt.tight_layout()
    if out_path:
        plt.savefig(out_path, dpi=500, bbox_inches="tight")
        plt.close()


def correct_for_site(data, area, site_col="site"):
    # Correct for site effects using linear regression
    site = OneHotEncoder(sparse_output=False, drop="first").fit_transform(
        data[[site_col]]
    )
    data[area] = data[area] - LinearRegression().fit(site, data[area]).predict(site)
    return data

In [ ]:
# Prepare Brain Imaging Data for PALM
corr_plot_path = op.join(WORKING_PATH, "palm_regensburg_cycle/plots")
os.makedirs(corr_plot_path, exist_ok=True)

cortical_areas = {
    "rostralanteriorcingulate": "Rostral Anterior Cingulate",
    "caudalanteriorcingulate": "Caudal Anterior Cingulate",
    "posteriorcingulate": "Posterior Cingulate",
    "parahippocampal": "Parahippocampal Gyrus",
    "lateralorbitofrontal": "Lateral Orbitofrontal Gyrus",
    "medialorbitofrontal": "Medial Orbitofrontal Gyrus",
    "insula": "Insula",
    "precuneus": "Precuneus",
}

subcortical_areas = {
    "Thalamus-Proper": "Thalamus",
    "Caudate": "Caudate Nucleus",
    "Accumbens-area": "Nucleus Accumbens",
    "Putamen": "Putamen",
    "Hippocampus": "Hippocampus",
    "Amygdala": "Amygdala",
}

# Cortical Thickness
for hemi in ["lh", "rh"]:
    for area, area_name in cortical_areas.items():
        data = db[
            [
                "site",
                "sex",
                "increase",
                f"{hemi}_{area}_thickness",
            ]
        ]
        out_file = f"{corr_plot_path}/{hemi}_{area}_thickness_increase.png"
        if op.exists(out_file):
            continue
        plot_brain_correlations(
            data,
            "increase",
            f"{hemi}_{area}_thickness",
            f"{area_name} thickness (mm)",
            out_file,
        )

# Surface Area
for hemi in ["lh", "rh"]:
    for area, area_name in cortical_areas.items():
        data = db[
            [
                "site",
                "sex",
                "increase",
                f"{hemi}_{area}_area",
            ]
        ]
        out_file = f"{corr_plot_path}/{hemi}_{area}_area_increase.png"
        if op.exists(out_file):
            continue
        plot_brain_correlations(
            data,
            "increase",
            f"{hemi}_{area}_area",
            f"{area_name} surface area (mm²)",
            out_file,
        )

# Subcortical Volume
for hemi in ["Left", "Right"]:
    for area, area_name in subcortical_areas.items():
        area_ = f"{hemi}-{area}"
        data = db[
            [
                "site",
                "sex",
                "increase",
                area_,
            ]
        ]
        out_file = f"{corr_plot_path}/{hemi}_{area}_volume_increase.png"
        if op.exists(out_file):
            continue
        plot_brain_correlations(
            data,
            "increase",
            area_,
            f"{area_name} volume (mm³)",
            out_file,
        )